## Тестировал в google colaboratory

In [1]:
import torch
from torch import nn
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from mpl_toolkits.mplot3d import Axes3D
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split

In [2]:
def f(x1, x2):
    return np.sin(x1 + 2*x2)* np.exp(-(2*x1 + x2)**2)

X1 = np.outer(np.linspace(-10, 10, 200), np.ones(200))
X2 = X1.copy().T

y = f(X1, X2)

In [3]:
import plotly.graph_objects as go

fig = go.Figure(data=[go.Surface(x=X1, y=X2, z=y)])
fig.show()

In [4]:
X1 = np.linspace(-10, 10, 20000)
X2 = X1.copy()

np.random.shuffle(X1), np.random.shuffle(X2) 

y = f(X1, X2)

df = pd.DataFrame({'X1': X1, 'X2': X2, 'y': y})

In [5]:
train = df.sample(frac=0.7, random_state=123)
indx = train.index
X_test, X_val, y_test, y_val = train_test_split(df.loc[~df.index.isin(indx), ['X1', 'X2']],
                                                    df.loc[~df.index.isin(indx), 'y'], random_state=123, test_size=0.5)

X_train, y_train = train[['X1', 'X2']], train['y']
y_train += np.random.normal(0, 0.1)  # noise

In [6]:
class Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(2, 200)
        self.fc2 = nn.Linear(200, 200)
        self.fc3 = nn.Linear(200, 200)
        self.fc4 = nn.Linear(200, 1)

    def forward(self, x):
        x = ReLU(self.fc1(x))
        x = ReLU(self.fc2(x))
        x = ReLU(self.fc3(x))
        return self.fc4(x)


class MyDataset(Dataset):
  
    def __init__(self, X, y):
        self.X = torch.Tensor(X.values)
        self.y = torch.from_numpy(y.values).float()

    def __len__(self):
        return self.X.shape[0]
  
    def __getitem__(self, index):
        return (self.X[index], self.y[index])

ds_train, ds_test, ds_val = MyDataset(X_train, y_train), MyDataset(X_test, y_test), MyDataset(X_val, y_val)
dl_train, dl_test, dl_val = DataLoader(ds_train, batch_size=64, shuffle=True), DataLoader(ds_test, batch_size=64), DataLoader(ds_val, batch_size=64)

In [12]:
model = Model()

In [13]:
ReLU = nn.ReLU()
loss_func = nn.MSELoss()
device = "cuda" if torch.cuda.is_available() else "cpu"
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, betas=(0.9, 0.999))

def train(dl, epoch):
    model.train()
    for ep in range(epoch):
        print(f'Epoch {ep+1} \n -------------------')
        for batch, (X, y) in enumerate(dl):
            X, y = X.to(device), y.to(device)

            # Делаем предсказания:
            pred = model(X)
            loss = loss_func(pred.squeeze(), y)

            # Backpropagation:
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            if batch % 50 == 0:
                loss, current = loss.item(), batch * len(X)
                print(f"loss: {loss:>7f}  [{current:>5d}/{len(ds_train):>5d}]")

        preds, true = test(dl_test)
        print(f'MSE = {loss_func(preds.squeeze(), true).item()}\n')
        

def test(dl):
    model.eval()
    preds, true = torch.empty(0), torch.empty(0)
    with torch.no_grad():
        for batch, (X, y) in enumerate(dl):
            X, y = X.to(device), y.to(device)

            # Делаем предсказания:
            pred = model(X)
            preds = torch.cat([preds, pred], 0)
            true = torch.cat([true, y], 0)
    return preds, true
    


In [14]:
train(dl_train, 30)

Epoch 1 
 -------------------
loss: 0.063066  [    0/14000]
loss: 0.019583  [ 3200/14000]
loss: 0.004730  [ 6400/14000]
loss: 0.001615  [ 9600/14000]
loss: 0.002331  [12800/14000]
MSE = 0.0193488746881485

Epoch 2 
 -------------------
loss: 0.014699  [    0/14000]
loss: 0.013483  [ 3200/14000]
loss: 0.021122  [ 6400/14000]
loss: 0.008651  [ 9600/14000]
loss: 0.014069  [12800/14000]
MSE = 0.018182789906859398

Epoch 3 
 -------------------
loss: 0.016484  [    0/14000]
loss: 0.011409  [ 3200/14000]
loss: 0.012376  [ 6400/14000]
loss: 0.024728  [ 9600/14000]
loss: 0.009569  [12800/14000]
MSE = 0.01797453500330448

Epoch 4 
 -------------------
loss: 0.001760  [    0/14000]
loss: 0.007457  [ 3200/14000]
loss: 0.018049  [ 6400/14000]
loss: 0.013340  [ 9600/14000]
loss: 0.005315  [12800/14000]
MSE = 0.015875859186053276

Epoch 5 
 -------------------
loss: 0.001517  [    0/14000]
loss: 0.007277  [ 3200/14000]
loss: 0.017006  [ 6400/14000]
loss: 0.007142  [ 9600/14000]
loss: 0.001155  [1280

In [15]:
preds, true = test(dl_val)
print(f'val MSE = {loss_func(preds.squeeze(), true).item()}')
X_val['y'] = preds
X_val['true'] = true
#X_val.sort_values('X1', inplace=True)

true_curve = pd.DataFrame({'X1': np.linspace(-5, 5, 2000), 'X2': -2*np.linspace(-5, 5, 2000)})
true_curve['y'] = f(true_curve['X1'], true_curve['X2'])

val MSE = 0.006757289171218872


In [16]:
import plotly.express as px

fig = go.Figure()

fig.add_trace(px.scatter_3d(X_val, x='X1', y='X2', z='y', color='y').data[0])
fig.add_trace(px.line_3d(true_curve, x="X1", y="X2", z="y").data[0])

fig.update_traces(marker_size = 5)
fig.show()